In [1]:
pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.5 MB/s eta 0:00:00m eta 0:00:010:00:02
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=2.0 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.5 MB/s eta 0:00:00
  Obtaining dependency information for grpcio<2.0,>=1.24.3 from https://files.pythonhosted.org/packages/3d/5b/0ae95c2817e0c9a510581a84c1a029fd447c7d0f289cc8b753e6ad93c216/grpcio-1.56.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 8.6 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.8 MB/s eta 0:00:00m eta 0:00:010:01:01
  Obtaining dependency information fo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 7.6 MB/s eta 0:00:00
  Obtaining dependency information for urllib3<2.0 from https://files.pythonhosted.org/packages/c5/05/c214b32d21c0b465506f95c4f28ccbcba15022e000b043b72b3df7728471/urllib3-1.26.16-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.6 MB/s eta 0:00:00
  Obtaining dependency information for importlib-metadata>=4.4 from https://files.pythonhosted.org/packages/ff/94/64287b38c7de4c90683630338cf28f129decbba0a44f0c6db35a873c73c4/importlib_metadata-6.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/89/f5/88e9dd454756fea555198ddbe6fa40d6408ec4f10ad4f0a911e0b7e471e4/charset_normalizer-3.2.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 7.7 MB/s eta 0:00:00
  Obtaining dependency information for MarkupSafe>=2.1.1

In [2]:
pip install pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 8.8 MB/s eta 0:00:000:00:01m eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
test = 'abcde'
test.replace('d','')

'abce'

In [61]:
import os
import json
import shutil
import random
import numpy as np
from PIL import ImageFile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [109]:
def load_dataset_from_folders(image_folder, json_folder, num_samples=1000):
    image_paths = []
    labels = []

    for food_name in os.listdir(image_folder):
        if food_name.startswith('.'):
            continue
        
        food_image_folder = os.path.join(image_folder, food_name)
        food_json_folder = os.path.join(json_folder, food_name)
        
        image_files = [file_name for file_name in os.listdir(food_image_folder) if file_name.endswith('.jpg')]

        sampled_images = random.sample(image_files, min(num_samples, len(image_files)))

        for file_name in sampled_images:
            image_path = os.path.join(food_image_folder, file_name)
            image_name = os.path.splitext(file_name)[0]

            json_file_path = os.path.join(food_json_folder + ' json/', image_name + '.json')

            with open(json_file_path, 'r') as f:
                annotation = json.load(f)

            label = annotation[0]['Name']
            label = label.replace('-','')
            if label is not None:
                if label == 'bogsung-a':
                    labels.append('peach')
                elif label == 'gamjaseupeu':
                    labels.append('potato_soup')
                else:
                    labels.append(label)
            image_paths.append(image_path)

    return image_paths, labels

In [123]:
# image_folder = './Training/[원천]음식203_Tra/'
# json_folder = './Training/[라벨]음식203_Tra_json/'
image_folder = './fooddata/train/source/'
json_folder = './fooddata/train/label/'

image_paths, labels = load_dataset_from_folders(image_folder, json_folder)

In [124]:
unique_labels = np.unique(labels)
unique_labels

array(['apple', 'bab', 'baechugimchi', 'banana', 'bangultomato',
       'beulokolli', 'biennasosiji', 'boiled_sweet_potatoes', 'chamoe',
       'chikinneoges', 'dalggalbi', 'dalgyalhulai', 'dalgyaljjim',
       'danhobag', 'doenjanggug', 'doenjangjjigae', 'eomuggug',
       'galbijjim', 'gamjabokkeum', 'gochujangajji', 'gugsu',
       'haemulbokkeum', 'hunjeyeoneo', 'jabchae', 'jomigim', 'jug',
       'kongnamul', 'korean_style_pancake', 'mandugug', 'miyeoggug',
       'miyeognamul', 'mugimchi', 'oi', 'oliloseugui', 'omeulles',
       'papeulika', 'podo', 'ppang', 'roasted_fish', 'saendeuwichi',
       'saengseongui', 'sagwa', 'salmeundalgyal', 'samgyeobsalgui',
       'sangchu', 'scrambled_egg', 'seukeulaembeuldeuegeu',
       'sundubujjigae', 'sunsaljjimdalg', 'tomato', 'tteoggug', 'uyu',
       'vienna_sausage', 'yachaebokkeum', 'yangbaechussam',
       'yangsangchusaelleodeu'], dtype='<U21')

In [125]:
label_mapping = {"gamjabokkeum":"감자볶음",
"gochujangajji":"고추 장아찌",
"chikinneoges":"치킨너겟",
"galbijjim":"갈비찜",
"mandugug":"만두국",
"oliloseugui":"오리구이",
"dalgyalhulai":"달걀후라이",
"gugsu":"국수",
"danhobag":"단호박찜",
"dalgyalbokkeumbab":"볶음밥",
"dalgyaljjim":"달걀찜",
"dalgyalhulai":"달걀후라이",
"dalggalbi":"닭갈비",
"doenjanggug":"된장국",
"doenjangjjigae":"된장찌개",
"tteoggug":"떡국",
"mugimchi":"무김치",
"miyeoggug":"미역국",
"miyeognamul":"미역나물",
"banana":"바나나",
"bab":"밥",
"bangultomato":"방울토마토",
"baechugimchi":"배추김치",
"korean_style_pancake":"부침개",
"beulokolli":"브로콜리",
"biennasosiji":"비엔나 소시지",
"vienna_sausage":"비엔나 소시지",
"ppang":"빵",
"sagwa":"사과",
"apple":"사과",
"boiled_sweet_potatoes":"고구마",
"salmeundalgyal":"삶은 달걀",
"samgyeobsalgui":"삼겹살구이",
"sangchu":"상추",
"saendeuwichi":"샌드위치",
"roasted_fish":"생선구이",
"saengseongui":"생선구이",
"sundubujjigae":"순두부찌개",
"sunsaljjimdalg":"찜닭",
"scrambled_egg":"스크램블 에그",
"seukeulaembeuldeuegeu":"스크램블 에그",
"yachaebokkeum":"야채볶음",
"yangbaechussam":"양배추쌈",
"yangsangchusaelleodeu":"양상추샐러드",
"eomuggug":"어묵국",
"omeulles":"오믈렛",
"oi":"오이",
"uyu":"우유",
"jabchae":"잡채",
"jomigim":"조미김",
"jug":"죽",
"chamoe":"참외",
"kongnamul":"콩나물 무침",
"tomato":"토마토",
"papeulika":"파프리카",
"podo":"포도",
"haemulbokkeum":"해물볶음",
"hunjeyeoneo":"훈제연어",
"hunjeoli":"훈제오리"}
len(label_mapping)
# label_mapping = {'galbigu-i': '갈비구이', 'gamjasaelleodeu': '감자샐러드', 'ganjang': '간장', 'potato_soup': '감자스프',
#                  'gamjagu-i': '감자구이','gamjabokk-eum':'감자볶음','gam':'감','tangerine_juice':'감귤주스','gacheudong':'가츠동',
#                  'gaji':'가지구이', 'gamjageulatang':'감자그라탕'}

58

In [126]:
korean_labels = [label_mapping[label] for label in labels]

unique_korean_labels = np.unique(korean_labels)
label_to_int = {label: i for i, label in enumerate(unique_korean_labels)}
int_labels = np.array([label_to_int[label] for label in korean_labels])

In [128]:
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(image_paths, int_labels, test_size=0.2, random_state=42)

In [129]:
import requests
requests.packages.urllib3.disable_warnings()
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [130]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(unique_korean_labels), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [131]:
batch_size = 32
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    preprocessing_function = preprocess_input
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

In [133]:
# colab용
# train_dir = '/content/drive/MyDrive/final/train'
# val_dir = '/content/drive/MyDrive/final/val'
# jupyter용
train_dir = './food/final2/train'
val_dir = './food/final2/val'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Copy train images to the train directory
for image_path in train_image_paths:
    directory_path = os.path.dirname(image_path)
    class_name = os.path.basename(directory_path)
    destination_dir = os.path.join(train_dir, class_name)
    os.makedirs(destination_dir, exist_ok=True)
    shutil.copy(image_path, destination_dir)

# Copy validation images to the validation directory
for image_path in val_image_paths:
    directory_path = os.path.dirname(image_path)
    class_name = os.path.basename(directory_path)
    destination_dir = os.path.join(val_dir, class_name)
    os.makedirs(destination_dir, exist_ok=True)
    shutil.copy(image_path, destination_dir)

In [134]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (299, 299),
    batch_size = batch_size,
    class_mode = 'sparse',
    classes = unique_korean_labels.tolist(),
    shuffle = True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = (299, 299),
    batch_size = batch_size,
    class_mode = 'sparse',
    classes = unique_korean_labels.tolist()
)

epochs = 10

Found 31410 images belonging to 52 classes.
Found 7903 images belonging to 52 classes.


In [135]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
model.fit(train_generator, validation_data=val_generator, epochs=epochs)

Epoch 1/10
982/982 [==============================] - 75327s 77s/step - loss: 0.8961 - accuracy: 0.7352 - val_loss: 0.5731 - val_accuracy: 0.8242
Epoch 2/10
982/982 [==============================] - 72278s 74s/step - loss: 0.5487 - accuracy: 0.8277 - val_loss: 0.5249 - val_accuracy: 0.8358
Epoch 3/10
982/982 [==============================] - 86167s 88s/step - loss: 0.4544 - accuracy: 0.8544 - val_loss: 0.4429 - val_accuracy: 0.8617
Epoch 4/10
982/982 [==============================] - 79695s 81s/step - loss: 0.4007 - accuracy: 0.8705 - val_loss: 0.4368 - val_accuracy: 0.8669
Epoch 5/10
982/982 [==============================] - 85547s 87s/step - loss: 0.3534 - accuracy: 0.8842 - val_loss: 0.4068 - val_accuracy: 0.8703
Epoch 6/10
982/982 [==============================] - 84805s 86s/step - loss: 0.3193 - accuracy: 0.8957 - val_loss: 0.3819 - val_accuracy: 0.8824
Epoch 7/10
982/982 [==============================] - 84046s 86s/step - loss: 0.2963 - accuracy: 0.9037 - val_loss: 0.3358 -

In [136]:
loss, accuracy = model.evaluate(val_generator)
print(f"Validation loss: {loss:.4f}, Validation accuracy: {accuracy:.4f}")

247/247 [==============================] - 17006s 69s/step - loss: 0.3318 - accuracy: 0.9019
Validation loss: 0.3318, Validation accuracy: 0.9019


In [137]:
model.save('./food_recognition_model2.h5')

In [138]:
model = tf.keras.models.load_model('./food_recognition_model2.h5')

In [139]:
# Step 9: Make Predictions on New Images
def predict_food_category(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.expand_dims(img, axis=0)
    img = preprocess_input(img)

    keys_with_value = []

    prediction = model.predict(img)[0]
    predicted_class_index = np.argmax(prediction)
    for key, value in label_to_int.items():
        if value == predicted_class_index:
            keys_with_value.append(key)
    return keys_with_value

In [140]:
src = []
name = []
test = []
image_dir = './food_test/'
for file in os.listdir(image_dir):
    if (file.find('.jpg') != -1):
        src.append(image_dir + file)
        name.append(file)
        test.append(predict_food_category(image_dir + file))


for i in range(len(test)):
    print(name[i] + " : , Predict : "+ str(test[i]))

1/1 [==============================] - 2s 2s/step
오리로스구이1.jpg : , Predict : ['해물볶음']
생선구이.jpg : , Predict : ['생선구이']
스크램블에그2.jpg : , Predict : ['밥']
오이2.jpg : , Predict : ['오이']
오이3.jpg : , Predict : ['오이']
콩나물3.jpg : , Predict : ['야채볶음']
치킨너겟1.jpg : , Predict : ['치킨너겟']
오리로스구이2.jpg : , Predict : ['빵']
치킨너겟3.jpg : , Predict : ['치킨너겟']
미역국1.jpg : , Predict : ['미역국']
죽3.jpg : , Predict : ['죽']
치킨너겟2.jpg : , Predict : ['치킨너겟']
바나나.jpg : , Predict : ['바나나']
배추김치1.jpg : , Predict : ['배추김치']
오리로스구이3.jpg : , Predict : ['야채볶음']
돼지갈비찜1.jpg : , Predict : ['갈비찜']
배추김치3.jpg : , Predict : ['배추김치']
생선구이2.jpg : , Predict : ['빵']
삼겹살구이2.jpg : , Predict : ['감자볶음']
포도1.jpg : , Predict : ['포도']
삼겹살구이.jpg : , Predict : ['무김치']
콩나물1.jpg : , Predict : ['야채볶음']
바나나2.jpg : , Predict : ['바나나']
달걀후라이2.jpg : , Predict : ['달걀후라이']
포도3.jpg : , Predict : ['포도']
방토.jpg : , Predict : ['방울토마토']
스크램블에그1.jpg : , Predict : ['밥']
미역국2.jpg : , Predict : ['미역국']
미역국3.jpg : , Predict : ['미역국']
달걀후라이3.jpg : , Predict

In [141]:
print(label_to_int)

{'갈비찜': 0, '감자볶음': 1, '고구마': 2, '고추 장아찌': 3, '국수': 4, '단호박찜': 5, '달걀찜': 6, '달걀후라이': 7, '닭갈비': 8, '된장국': 9, '된장찌개': 10, '떡국': 11, '만두국': 12, '무김치': 13, '미역국': 14, '미역나물': 15, '바나나': 16, '밥': 17, '방울토마토': 18, '배추김치': 19, '부침개': 20, '브로콜리': 21, '비엔나 소시지': 22, '빵': 23, '사과': 24, '삶은 달걀': 25, '삼겹살구이': 26, '상추': 27, '샌드위치': 28, '생선구이': 29, '순두부찌개': 30, '스크램블 에그': 31, '야채볶음': 32, '양배추쌈': 33, '양상추샐러드': 34, '어묵국': 35, '오리구이': 36, '오믈렛': 37, '오이': 38, '우유': 39, '잡채': 40, '조미김': 41, '죽': 42, '찜닭': 43, '참외': 44, '치킨너겟': 45, '콩나물 무침': 46, '토마토': 47, '파프리카': 48, '포도': 49, '해물볶음': 50, '훈제연어': 51}
